In [1]:
%matplotlib inline
import math
import time
import json
import pandas as pd
import os
import numpy as np
import sqlite3
import boto3
import tensorflow as tf
import matplotlib as plt

In [2]:
from logfuncts import logger
from modelLib import rnn_model, load_dataframe_from_sql


FDIR = os.path.abspath(os.path.dirname("."))
DATABASE_PATH = os.path.join(FDIR, '../data.db')
OUTPUT_PATH = os.path.join(FDIR, '../html/dart.json')
RIVER_NAME = "dart"

MIMIMUM_THRESHOLD = 0.7
MAXIMUM_THRESHOLD = 1.5

In [5]:
testing_mode = True
testing_timestamp = "2019-04-07 21:30:00"

output = rnn_model(testing_mode, testing_timestamp)

2019-04-26 19:32:19,123 root         DEBUG    loading entire df for river dart from sql
2019-04-26 19:32:19,690 root         DEBUG    latest_level_update_timestamp: 2019-04-06 21:30:00
2019-04-26 19:32:19,692 root         DEBUG    latest_rain_time: 2019-04-07 21:00:00
2019-04-26 19:32:19,693 root         DEBUG    latest_forecast_rain_time: 2019-04-08 21:00:00
2019-04-26 19:32:19,702 root         DEBUG    num_level_updates: 96
2019-04-26 19:32:19,704 root         DEBUG    num_rain_updates: 190
2019-04-26 19:32:19,706 root         DEBUG    num_forecast_rain_updates: 286


INFO:tensorflow:Restoring parameters from /Users/jconn/isthedartrunning/python/production_rnn/variables/variables


2019-04-26 19:32:19,975 tensorflow   INFO     Restoring parameters from /Users/jconn/isthedartrunning/python/production_rnn/variables/variables
2019-04-26 19:32:20,182 root         INFO     currenct level: 0.374000012875
2019-04-26 19:32:20,185 root         INFO     OUTPUT TEXT: NO


In [6]:
with open("../html/dart.json", 'w') as f:
    json.dump(output, f, indent=4)